In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import scanpy as sc

import glob
import os
import scrna
import h5py

from micron2 import load_as_anndata
from micron2 import cluster_leiden_cu

import tqdm.auto as tqdm

from matplotlib import pyplot as plt
from matplotlib import rcParams
import time

from micron2.data import staining_border_nonzero

In [ ]:
datasets = !ls -lha /storage/codex/preprocessed_data/*/*_nosubtract.hdf5 | awk '{print $9}'
datasets = [d for d in datasets if 'reg0' not in d]
datasets

In [ ]:
len(datasets)

In [ ]:
ring_channels = ['CD45', 'CD20', 'CD3e', 'CD45RO', 'CD45RA', 'CD8', 'CD4']
adatas = []
sample_ids = []
for path in datasets:
    try:
        ad = load_as_anndata(path, 
                             recover_tile_nuclei=False, 
                             as_sparse = False,
                             features_dtype = None
                            )
    except:
        print('failed to load', path)
        continue
    
#     # Apply staining border function
#     with h5py.File(ad.uns['source_data'], 'r') as h5f:
#         ncells = ad.shape[0]
#         h5_ncells = h5f['cell_intensity']['DAPI'][:].shape[0]
#         print(ncells, h5_ncells)
#         ring_positive_pct = pd.DataFrame(index=ad.obs_names,
#                                          columns=[f'{ch}_ringpct' for ch in ring_channels],
#                                          dtype=np.float32
#                                         )
#         tstart = time.time()
#         for i in tqdm.trange(ncells):
#             m = h5f['meta']['nuclear_masks'][i,...]
#             vect = []
#             for ch in ring_channels:
#                 x = h5f['cells'][ch][i,...]
#                 v = staining_border_nonzero(x,m)
#                 vect.append(v)
#             ring_positive_pct.loc[ad.obs_names[i],:] = vect
#         tend = time.time()
#         print(f'elapsed time: {tend-tstart:3.4f}s')
#     ad.obs = pd.concat([ad.obs, ring_positive_pct], axis=1)

    
    adatas.append(ad.copy()) 
    s = os.path.splitext(os.path.basename(path))[0].replace('_nosubtract', '')
    sample_ids.append(s)

adata = adatas[0].concatenate(adatas[1:], batch_key='sample_id', batch_categories=sample_ids, 
                              index_unique = '-')
print(sample_ids)
adata.raw = adata


In [ ]:
print(len(sample_ids))
sample_ids

In [ ]:
sample_ids_padded = sample_ids
while len(sample_ids_padded) % 2 != 0:
    sample_ids_padded.append('pass')
sample_layout = np.expand_dims(np.array(sample_ids_padded), axis=0).reshape(2,-1)
# sample_layout = np.expand_dims(np.array(sample_ids+['pass']), axis=0).reshape(2,-1)
print(np.array(sample_ids).shape)
print(sample_layout.shape)
print(sample_layout)

In [ ]:
sample_id_printing = [x.split('Breast_')[1].replace('_','\n') for x in adata.obs.sample_id]
adata.obs['sample_id_printing'] = sample_id_printing
adata.uns['channels'] = ad.uns['channels']

In [ ]:
adata

In [ ]:
mean_features = [v for v in adata.var_names if 'mean' in v]
adata = adata[:,mean_features]
adata

In [ ]:
!ls -lha /storage/codex/datasets_v1/

In [ ]:
adata.write("/storage/codex/datasets_v1/merged_v3_nosubtract.h5ad")

In [ ]:
!ls -lha /storage/codex/datasets_v1/